In [ ]:
import os

os.chdir('..')

In [ ]:
import polars as pl
from config import BINANCE_DATA_DIR, TradeType
from generate.merge import merge_klines, merge_funding_rates
from util.time import convert_interval_to_timedelta

In [ ]:
symbol = 'BTCUSDT'
trade_type = TradeType.spot
    
kline_dir = BINANCE_DATA_DIR / 'parsed_data' / trade_type.value / 'klines' / symbol / '1m'
ldf = pl.scan_parquet(kline_dir).sort('candle_begin_time')
df = ldf.collect()
df.flags

In [ ]:
time_interval = '1m'

complete_times = pl.datetime_range(
    df["candle_begin_time"].min(),
    df["candle_begin_time"].max(),
    interval=convert_interval_to_timedelta(time_interval),
    time_zone="UTC",
    time_unit=df["candle_begin_time"].dtype.time_unit,
    eager=True,
)

# Create template df with all timestamps
template_df = pl.LazyFrame({"candle_begin_time": complete_times})
template_df.show_graph() 

In [ ]:
# Join with original data
ldf = template_df.join(df.lazy(), on="candle_begin_time", how="left")

ldf = ldf.with_columns(pl.col("close").fill_null(strategy="forward"))
ldf = ldf.with_columns(
    pl.col("open").fill_null(pl.col("close")),
    pl.col("high").fill_null(pl.col("close")),
    pl.col("low").fill_null(pl.col("close")),
)
# Fill volumes with 0
ldf = ldf.with_columns(
    pl.col("volume").fill_null(0),
)

ldf.show_graph()